# Motion Capture Data Preprocessing

This notebook performs preprocessing on the motion capture data, i.e., removing postitional and rotational drifts.

## Imports and Global Helper Functions

In [ ]:
import json

from pathlib import Path

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import biopsykit as bp
from fau_colors import cmaps

from empkins_io.sensors.motion_capture.perception_neuron import PerceptionNeuronDataset
from stresspose_analysis.data_processing.perception_neuron import process_bvh, process_calc, process_center_of_mass

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
plt.close("all")

palette = cmaps.faculties
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams["figure.figsize"] = (8, 4)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

sns.color_palette(palette)

In [ ]:
def plot_data(data: pd.DataFrame, channel: str = None, body_part: str = "Hips", ax: plt.Axes = None):
    if ax is None:
        fig, ax = plt.subplots()
    else:
        fig = ax.get_figure()
    if channel is None:
        data.loc[:, pd.IndexSlice[body_part, :, :]].plot(ax=ax)
    else:
        data.loc[:, pd.IndexSlice[body_part, channel, :]].plot(ax=ax)
    fig.tight_layout()

## Variable and Path Setup

In [ ]:
deploy_type = "local"

In [ ]:
config_dict = json.load(Path("../../config.json").open(encoding="utf-8"))

base_path = Path(config_dict[deploy_type]["base_path"])
data_path = base_path.joinpath("data_per_subject")

subject_dirs = bp.utils.file_handling.get_subject_dirs(data_path, "VP_*")
print(f"Found {len(subject_dirs)} subjects.")

In [ ]:
subject_id = "VP_02"
condition = "tsst"

In [ ]:
mocap_path = data_path.joinpath(f"{subject_id}/{condition}/mocap")
input_path = mocap_path.joinpath("export")
output_path = mocap_path.joinpath("filtered")
bp.utils.file_handling.mkdirs(output_path)

In [ ]:
bvh_file = f"{subject_id}_{condition}.bvh.gz"
calc_file = f"{subject_id}_{condition}.calc.gz"
global_pose_file = f"{subject_id}_{condition}_global_pose.csv.gz"

bvh_path = input_path.joinpath(bvh_file)
calc_path = input_path.joinpath(calc_file)
global_pose_path = input_path.joinpath(global_pose_file)

filter_params = json.load(mocap_path.joinpath(f"{subject_id}_filter_params_{condition}.json").open())
filter_params

In [ ]:
dataset = PerceptionNeuronDataset.from_folder(input_path, **filter_params["start_end"])
data_dict = dataset.data_dict

In [ ]:
pos_filter_params = filter_params["pos_filter_params"]
rot_filter_params = filter_params["rot_filter_params"]

## Filter Motion Capture Data

### BVH Data

In [ ]:
bvh_proc = process_bvh(data_dict, pos_filter_params, rot_filter_params)

In [ ]:
bvh_data = bvh_proc.data_dict["rot"]
bvh_data_global = bvh_proc.data_dict["global_pose"]
bvh_data.data.head()

### Calc Data

In [ ]:
calc_proc = process_calc(data_dict, pos_filter_params, rot_filter_params)

In [ ]:
calc_data = calc_proc.data_dict["rot"]
calc_data.data.head()

### Center of Mass Data

In [ ]:
center_mass_proc = process_center_of_mass(data_dict, pos_filter_params)

In [ ]:
center_mass_data = center_mass_proc.data_dict["pos"]
center_mass_data.data.head()

## Export

In [ ]:
bvh_data.to_gzip_bvh(output_path.joinpath(f"{subject_id}_{condition}.bvh.gz"))
bvh_data.global_pose_to_gzip_csv(output_path.joinpath(f"{subject_id}_{condition}_global_pose.csv.gz"))

In [ ]:
calc_data.to_gzip_calc(output_path.joinpath(f"{subject_id}_{condition}.calc.gz"))

In [ ]:
center_mass_data.to_csv(output_path.joinpath(f"{subject_id}_{condition}_centerOfMass.csv"))